In [ ]:
import skiba.data_process as sdp
import ipywidgets as widgets
import geopandas as gpd
import requests
import json

In [ ]:
url = (
    "https://github.com/opengeos/geospatial-data-catalogs/blob/master/gee_catalog.json"
)

In [ ]:
# def fetch_geojson(url):
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Raises an exception for HTTP errors
#         geojson_data = response.json()  # Parse the JSON response
#         return geojson_data
#     except requests.exceptions.HTTPError as http_err:
#         print(f"HTTP error occurred: {http_err}")
#     except requests.exceptions.ConnectionError as conn_err:
#         print(f"Error connecting to the server: {conn_err}")
#     except Exception as err:
#         print(f"An error occurred: {err}")
#     return None

# geojson_data = fetch_geojson(url)


data = "../data/gee_catalog.json"
with open(data, "r") as file:
    data = json.load(file)

data_dict = {item["title"]: item["id"] for item in data if "title" in item}

# Step 4: Create the dropdown
dropdown = widgets.Dropdown(
    options=data_dict,  # keys shown, values returned
    description="Dataset:",
    disabled=False,
)

display(dropdown)

In [ ]:
dropdown.value

In [ ]:
points = "../data/fuzzy_coordinates"

In [ ]:
m = sdp.data_process
dropdown_widget = m.create_dropdown()
dropdown_widget

In [ ]:
dropdown_widget.value

In [ ]:
sdp.create